# BioBatchNet 使用教程

BioBatchNet 是一个用于生物数据批次效应矫正的深度学习框架，支持单细胞RNA测序(scRNA-seq)和成像质谱流式(IMC)数据。

本教程将介绍：
1. 快速开始 - 使用简单API
2. 高级使用 - 直接使用模型
3. 自定义配置 - 调整模型架构和训练参数
4. 实际案例演示

## 1. 安装和导入

In [ ]:
# 安装包 (如果还没安装)
# !pip install biobatchnet

# 或从源码安装
# !git clone https://github.com/Manchester-HealthAI/BioBatchNet
# !cd BioBatchNet && pip install -e .

In [ ]:
# 导入必要的包
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 导入BioBatchNet
import biobatchnet
from biobatchnet import correct_batch_effects, IMCVAE, GeneVAE

print(f"BioBatchNet版本: {biobatchnet.__version__}")
print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")

## 2. 准备示例数据

In [ ]:
# 生成模拟数据用于演示
np.random.seed(42)

# 模拟IMC数据: 1000个细胞，40个蛋白标记，3个批次
n_cells = 1000
n_features = 40
n_batches = 3

# 生成基础数据
base_data = np.random.randn(n_cells, n_features)

# 添加批次效应
batch_labels = np.random.choice(n_batches, n_cells)
batch_effects = np.zeros_like(base_data)
for i in range(n_batches):
    batch_mask = batch_labels == i
    # 每个批次添加不同的偏移
    batch_effects[batch_mask] = np.random.randn(1, n_features) * 0.5

# 最终数据 = 基础数据 + 批次效应
data_with_batch = base_data + batch_effects

# 转换为DataFrame
data_df = pd.DataFrame(
    data_with_batch, 
    columns=[f'Protein_{i+1}' for i in range(n_features)]
)

batch_df = pd.DataFrame({
    'batch_id': batch_labels,
    'cell_id': [f'cell_{i}' for i in range(n_cells)]
})

print(f"数据形状: {data_df.shape}")
print(f"批次分布: {np.bincount(batch_labels)}")

In [ ]:
# 可视化批次效应
def plot_batch_effect(data, batch_labels, title):
    """使用PCA可视化批次效应"""
    pca = PCA(n_components=2)
    data_pca = pca.fit_transform(StandardScaler().fit_transform(data))
    
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(data_pca[:, 0], data_pca[:, 1], 
                         c=batch_labels, cmap='viridis', alpha=0.6)
    plt.colorbar(scatter, label='Batch')
    plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%})')
    plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%})')
    plt.title(title)
    plt.show()

# 显示原始数据的批次效应
plot_batch_effect(data_with_batch, batch_labels, '原始数据 (含批次效应)')

## 3. 方法一：使用简单API (推荐)

这是最简单的使用方式，适合大多数用户。

In [ ]:
# 使用默认参数进行批次效应矫正
bio_embeddings, batch_embeddings = correct_batch_effects(
    data=data_df,           # 表达数据
    batch_info=batch_df,    # 批次信息
    batch_key='batch_id',   # 批次列名
    data_type='imc',        # 数据类型: 'imc' 或 'scrna'
    latent_dim=20,          # 潜在空间维度
    epochs=100              # 训练轮数
)

print(f"生物学嵌入形状: {bio_embeddings.shape}")
print(f"批次嵌入形状: {batch_embeddings.shape}")

In [ ]:
# 可视化矫正后的数据
plot_batch_effect(bio_embeddings, batch_labels, '矫正后的生物学嵌入')

### 3.1 自定义损失权重

根据数据特点调整各项损失的权重。

In [ ]:
# 自定义损失权重
custom_loss_weights = {
    'recon_loss': 10,      # 重建损失权重
    'discriminator': 0.3,   # 判别器损失权重
    'classifier': 1,        # 分类器损失权重
    'kl_loss_1': 0.005,    # KL散度损失1
    'kl_loss_2': 0.1,      # KL散度损失2
    'ortho_loss': 0.01     # 正交损失权重
}

bio_embeddings_custom, batch_embeddings_custom = correct_batch_effects(
    data=data_df,
    batch_info=batch_df,
    batch_key='batch_id',
    data_type='imc',
    latent_dim=20,
    epochs=100,
    loss_weights=custom_loss_weights  # 使用自定义权重
)

print("使用自定义损失权重完成训练")

### 3.2 不同批次数量的自动参数调整

API会根据批次数量自动调整参数。

In [ ]:
# 模拟不同批次数量的数据
def test_different_batch_counts():
    for n_batches in [3, 15, 35]:
        # 创建测试数据
        test_batch_labels = np.random.choice(n_batches, n_cells)
        test_batch_df = pd.DataFrame({'batch_id': test_batch_labels})
        
        print(f"\n批次数量: {n_batches}")
        print("API将自动选择合适的损失权重")
        
        # API会自动调整参数
        bio_emb, batch_emb = correct_batch_effects(
            data=data_df,
            batch_info=test_batch_df,
            data_type='imc',
            epochs=50  # 减少轮数以加快演示
        )
        
        print(f"完成! 嵌入维度: {bio_emb.shape}")

# test_different_batch_counts()  # 取消注释以运行

## 4. 方法二：直接使用模型 (高级)

直接使用模型类，获得更多控制权。

In [ ]:
# 创建IMCVAE模型实例
model = IMCVAE(
    in_sz=n_features,                              # 输入维度
    out_sz=n_features,                             # 输出维度
    latent_sz=20,                                  # 潜在空间维度
    num_batch=n_batches,                           # 批次数量
    bio_encoder_hidden_layers=[512, 1024, 1024],   # 生物编码器架构
    batch_encoder_hidden_layers=[256],             # 批次编码器架构
    decoder_hidden_layers=[1024, 1024, 512],       # 解码器架构
    batch_classifier_layers_power=[512, 1024, 1024], # 强分类器架构
    batch_classifier_layers_weak=[128]             # 弱分类器架构
)

print(f"模型参数数量: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# 训练模型
model.fit(
    data=data_df.values,
    batch_info=batch_labels,
    epochs=100,
    lr=1e-3,
    batch_size=256,
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

print("模型训练完成")

In [ ]:
# 获取矫正后的嵌入
bio_embeddings_direct = model.get_bio_embeddings(data_df.values)
print(f"生物学嵌入形状: {bio_embeddings_direct.shape}")

# 或者同时获取生物学和批次嵌入
bio_emb, batch_emb = model.correct_batch_effects(data_df.values)
print(f"生物学嵌入: {bio_emb.shape}, 批次嵌入: {batch_emb.shape}")

## 5. 方法三：使用GeneVAE处理scRNA-seq数据

GeneVAE专门设计用于单细胞RNA测序数据。

In [ ]:
# 模拟scRNA-seq数据
n_cells_rna = 5000
n_genes = 2000
n_batches_rna = 4

# 生成稀疏的基因表达数据（模拟dropout）
rna_data = np.random.negative_binomial(5, 0.3, size=(n_cells_rna, n_genes))
rna_data = rna_data.astype(np.float32)

# 添加批次效应
rna_batch_labels = np.random.choice(n_batches_rna, n_cells_rna)
for i in range(n_batches_rna):
    mask = rna_batch_labels == i
    rna_data[mask] *= np.random.uniform(0.8, 1.2)  # 批次特异性缩放

print(f"scRNA-seq数据形状: {rna_data.shape}")
print(f"零值比例: {(rna_data == 0).mean():.2%}")

In [ ]:
# 使用API处理scRNA-seq数据
bio_emb_rna, batch_emb_rna = correct_batch_effects(
    data=rna_data,
    batch_info=rna_batch_labels,
    data_type='scrna',  # 指定为scRNA-seq数据
    latent_dim=30,      # 通常scRNA需要更高的潜在维度
    epochs=100
)

print(f"scRNA嵌入形状: {bio_emb_rna.shape}")

In [ ]:
# 直接使用GeneVAE模型
gene_model = GeneVAE(
    in_sz=n_genes,
    out_sz=n_genes,
    latent_sz=30,
    num_batch=n_batches_rna,
    bio_encoder_hidden_layers=[500, 2000, 2000],   # 默认scRNA架构
    batch_encoder_hidden_layers=[500],
    decoder_hidden_layers=[2000, 2000, 500],
    batch_classifier_layers_power=[500, 2000, 2000],
    batch_classifier_layers_weak=[128]
)

# 自定义scRNA-seq的损失权重
scrna_loss_weights = {
    'recon_loss': 10,
    'discriminator': 0.04,
    'classifier': 1,
    'kl_loss_1': 1e-7,
    'kl_loss_2': 0.01,
    'ortho_loss': 0.0002,
    'mmd_loss_1': 0,
    'kl_loss_size': 0.002  # scRNA特有的size factor KL损失
}

gene_model.fit(
    data=rna_data,
    batch_info=rna_batch_labels,
    epochs=100,
    loss_weights=scrna_loss_weights
)

print("GeneVAE模型训练完成")

## 6. 高级配置和调参技巧

In [ ]:
# 高级配置示例
advanced_config = {
    # 数据参数
    'data': data_df,
    'batch_info': batch_df,
    'batch_key': 'batch_id',
    'data_type': 'imc',
    
    # 模型架构参数
    'latent_dim': 25,
    'bio_encoder_hidden_layers': [256, 512, 512],  # 自定义编码器架构
    'batch_encoder_hidden_layers': [128, 128],     # 两层批次编码器
    'decoder_hidden_layers': [512, 512, 256],      # 自定义解码器
    
    # 训练参数
    'epochs': 150,
    'lr': 5e-4,           # 学习率
    'batch_size': 128,    # 批次大小
    
    # 损失权重
    'loss_weights': {
        'recon_loss': 15,
        'discriminator': 0.2,
        'classifier': 1.5,
        'kl_loss_1': 0.001,
        'kl_loss_2': 0.05,
        'ortho_loss': 0.02
    }
}

# 使用高级配置
bio_emb_advanced, batch_emb_advanced = correct_batch_effects(**advanced_config)
print("高级配置训练完成")

## 7. 批次效应矫正效果评估

In [ ]:
from sklearn.metrics import silhouette_score
from scipy.stats import f_oneway

def evaluate_batch_correction(original_data, corrected_data, batch_labels):
    """
    评估批次效应矫正的效果
    """
    # 1. 轮廓系数 (越小表示批次混合越好)
    sil_original = silhouette_score(original_data, batch_labels)
    sil_corrected = silhouette_score(corrected_data, batch_labels)
    
    print(f"轮廓系数 (批次分离度，越小越好):")
    print(f"  原始数据: {sil_original:.4f}")
    print(f"  矫正后: {sil_corrected:.4f}")
    print(f"  改善: {(sil_original - sil_corrected) / sil_original * 100:.1f}%\n")
    
    # 2. ANOVA F统计量 (越小表示批次差异越小)
    groups_original = [original_data[batch_labels == i] for i in range(len(np.unique(batch_labels)))]
    groups_corrected = [corrected_data[batch_labels == i] for i in range(len(np.unique(batch_labels)))]
    
    # 计算前5个特征的F统计量
    f_stats_original = []
    f_stats_corrected = []
    
    n_features_to_test = min(5, original_data.shape[1])
    for i in range(n_features_to_test):
        f_orig, _ = f_oneway(*[g[:, i] for g in groups_original])
        f_corr, _ = f_oneway(*[g[:, i] for g in groups_corrected])
        f_stats_original.append(f_orig)
        f_stats_corrected.append(f_corr)
    
    print(f"平均F统计量 (批次间差异，越小越好):")
    print(f"  原始数据: {np.mean(f_stats_original):.4f}")
    print(f"  矫正后: {np.mean(f_stats_corrected):.4f}")
    print(f"  改善: {(np.mean(f_stats_original) - np.mean(f_stats_corrected)) / np.mean(f_stats_original) * 100:.1f}%")
    
    return sil_corrected, np.mean(f_stats_corrected)

# 评估矫正效果
print("=" * 50)
print("批次效应矫正效果评估")
print("=" * 50)
evaluate_batch_correction(data_with_batch, bio_embeddings, batch_labels)

## 8. 可视化对比

In [ ]:
# 创建对比可视化
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# 原始数据PCA
pca_original = PCA(n_components=2)
data_pca_original = pca_original.fit_transform(StandardScaler().fit_transform(data_with_batch))

# 矫正后数据PCA
pca_corrected = PCA(n_components=2)
data_pca_corrected = pca_corrected.fit_transform(StandardScaler().fit_transform(bio_embeddings))

# 绘制原始数据
scatter1 = axes[0].scatter(data_pca_original[:, 0], data_pca_original[:, 1],
                           c=batch_labels, cmap='viridis', alpha=0.6, s=20)
axes[0].set_xlabel(f'PC1 ({pca_original.explained_variance_ratio_[0]:.2%})')
axes[0].set_ylabel(f'PC2 ({pca_original.explained_variance_ratio_[1]:.2%})')
axes[0].set_title('原始数据 (含批次效应)')
axes[0].legend(*scatter1.legend_elements(), title="批次", loc="best")

# 绘制矫正后数据
scatter2 = axes[1].scatter(data_pca_corrected[:, 0], data_pca_corrected[:, 1],
                           c=batch_labels, cmap='viridis', alpha=0.6, s=20)
axes[1].set_xlabel(f'PC1 ({pca_corrected.explained_variance_ratio_[0]:.2%})')
axes[1].set_ylabel(f'PC2 ({pca_corrected.explained_variance_ratio_[1]:.2%})')
axes[1].set_title('BioBatchNet矫正后')
axes[1].legend(*scatter2.legend_elements(), title="批次", loc="best")

plt.suptitle('批次效应矫正前后对比', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

## 9. 保存和加载模型

In [ ]:
# 保存训练好的模型
model_path = 'biobatchnet_model.pt'
torch.save({
    'model_state_dict': model.state_dict(),
    'model_config': {
        'in_sz': n_features,
        'out_sz': n_features,
        'latent_sz': 20,
        'num_batch': n_batches,
        'bio_encoder_hidden_layers': [512, 1024, 1024],
        'batch_encoder_hidden_layers': [256],
        'decoder_hidden_layers': [1024, 1024, 512],
        'batch_classifier_layers_power': [512, 1024, 1024],
        'batch_classifier_layers_weak': [128]
    }
}, model_path)

print(f"模型已保存到 {model_path}")

In [ ]:
# 加载模型
checkpoint = torch.load(model_path, map_location='cpu')
config = checkpoint['model_config']

# 重新创建模型
loaded_model = IMCVAE(**config)
loaded_model.load_state_dict(checkpoint['model_state_dict'])
loaded_model.eval()

print("模型加载成功")

# 使用加载的模型进行预测
with torch.no_grad():
    bio_emb_loaded = loaded_model.get_bio_embeddings(data_df.values)
    print(f"加载模型的预测结果形状: {bio_emb_loaded.shape}")

## 10. 常见问题和调参建议

### 10.1 如何选择潜在维度（latent_dim）？
- IMC数据：通常15-25维
- scRNA-seq数据：通常20-50维
- 可以通过交叉验证选择最佳值

### 10.2 损失权重调整策略
- **recon_loss**: 重建质量，通常设为10
- **discriminator**: 批次混合程度，批次多时降低(0.1-0.3)
- **classifier**: 批次信息保留，通常设为1
- **kl_loss**: 正则化强度，过拟合时增加
- **ortho_loss**: 正交性约束，保持默认即可

### 10.3 训练不稳定怎么办？
- 降低学习率
- 减小batch_size
- 调整损失权重
- 增加训练轮数

### 10.4 内存不足怎么办？
- 减小batch_size
- 使用CPU训练：device='cpu'
- 降低模型复杂度（减少隐藏层节点数）

## 11. 实际数据示例（使用AnnData）

In [ ]:
# 如果有实际的h5ad文件
import anndata as ad

# 示例：加载和处理实际数据
"""
# 加载AnnData对象
adata = ad.read_h5ad('your_data.h5ad')

# 提取数据和批次信息
if hasattr(adata.X, 'toarray'):
    X = adata.X.toarray()  # 如果是稀疏矩阵
else:
    X = adata.X

batch_labels = adata.obs['batch'].values

# 批次效应矫正
bio_embeddings, _ = correct_batch_effects(
    data=X,
    batch_info=batch_labels,
    data_type='scrna',  # 或 'imc'
    epochs=200
)

# 将结果存回AnnData
adata.obsm['X_biobatchnet'] = bio_embeddings

# 保存结果
adata.write('corrected_data.h5ad')
"""

print("实际数据处理流程示例（需要提供h5ad文件）")

## 总结

BioBatchNet提供了灵活的批次效应矫正方案：

1. **简单API** (`correct_batch_effects`): 适合快速使用，自动参数选择
2. **模型类** (`IMCVAE`, `GeneVAE`): 提供更多控制，适合高级用户
3. **自定义配置**: 可以调整模型架构、损失权重等参数

### 主要特点：
- 支持IMC和scRNA-seq两种数据类型
- 自动根据批次数量调整参数
- 可自定义模型架构和训练参数
- 提供生物学和批次两种嵌入输出

### 最佳实践：
1. 先使用默认参数尝试
2. 根据结果调整损失权重
3. 必要时调整模型架构
4. 使用评估指标验证效果

更多信息请参考：
- GitHub: https://github.com/Manchester-HealthAI/BioBatchNet
- 文档: USAGE.md